In [ ]:
!pip install gradio

In [ ]:
!git clone https://github.com/Keerthiga2005-S/ML--Projects.git

Cloning into 'ML--Projects'...


In [ ]:
import pickle


In [ ]:
import pandas as pd

In [ ]:
d=pd.read_csv("/content/healthcare-dataset-stroke-data.csv")

In [ ]:
d=d.drop("bmi",axis=1)

In [ ]:
d=d.drop('ever_married',axis=1)

In [ ]:
d=d.drop('id',axis=1)

In [ ]:
d=d.drop('gender',axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
d['Residence_type']=le.fit_transform(d['Residence_type'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
d['work_type']=le.fit_transform(d['work_type'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
d['smoking_status']=le.fit_transform(d['smoking_status'])

In [ ]:
x=d.iloc[:,:-1]
y=d.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

0.9412915851272016


In [ ]:
with open("stroke.pkl", "wb") as f:
    pickle.dump(model, f)


In [ ]:
with open("stroke.pkl", "rb") as f:
	model = pickle.load(f)


In [ ]:
import gradio as gr

In [ ]:
import gradio as gr
import pickle
import numpy as np

def preprocess_input(age, hypertension, heart_disease, work_type, Residence_type, avg_glucose_level, smoking_status):
    # Encode categorical variables
    work_type_encoded = {"Private": 0, "Self-employed": 1, "Govt_job": 2, "children": 3, "Never_worked": 4}[work_type]
    Residence_type_encoded = {"Urban": 0, "Rural": 1}[Residence_type]
    smoking_status_encoded = {"formerly smoked": 0, "never smoked": 1, "smokes": 2, "Unknown": 3}[smoking_status]


    return np.array([age, hypertension, heart_disease, work_type_encoded, Residence_type_encoded, avg_glucose_level, smoking_status_encoded]).reshape(1, -1)

def make_prediction(age, hypertension, heart_disease, work_type, Residence_type, avg_glucose_level, smoking_status):
    try:
        # Load the saved model
        with open("stroke.pkl", "rb") as f:
            model = pickle.load(f)

        # Preprocess input
        input_features = preprocess_input(age, hypertension, heart_disease, work_type, Residence_type, avg_glucose_level, smoking_status)

        # Make prediction using the loaded model
        preds = model.predict(input_features)

        # Return prediction result
        if preds == 1:
            return "High stroke risk"
        else:
            return "Low stroke risk"
    except Exception as e:
        return f"Prediction error: {str(e)}"

# Create Gradio input components
age_input = gr.Number(label="Enter Age")
hypertension_input = gr.Number(label="Hypertension (0: No, 1: Yes)")
heart_disease_input = gr.Number(label="Heart Disease (0: No, 1: Yes)")
work_type_input = gr.Radio(label="Work Type", choices=["Private", "Self-employed", "Govt_job", "children", "Never_worked"])
Residence_type_input = gr.Radio(label="Residence Type", choices=["Urban", "Rural"])
avg_glucose_level_input = gr.Number(label="Average Glucose Level")
smoking_status_input = gr.Radio(label="Smoking Status", choices=["formerly smoked", "never smoked", "smokes", "Unknown"])

# Create the output component
output = gr.Textbox()

# Set up the Gradio interface
app = gr.Interface(fn=make_prediction,
                   inputs=[age_input, hypertension_input, heart_disease_input, work_type_input,
                           Residence_type_input, avg_glucose_level_input, smoking_status_input],
                   outputs=output)

# Launch the Gradio interface
app.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://edbe078ffbdf04baa0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
